# Extract all mooc-list course website URLs

In [1]:
# Handle imports and set SSL

from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
from datetime import datetime
import pickle
import ssl
import requests
import re
import json
import csv

ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

### Before that, we need to extract all universities URLs

In [2]:
def extract_univ_urls(univ_url) :
	urls_level_1 = []

	try:
		html_req = Request(univ_url, headers={'User-Agent': 'Mozilla/5.0'})
		html = urlopen(html_req, context=ctx).read()
	except HTTPError as e:
		print(e)
		return None 

	try:
		soup = BeautifulSoup(html, 'html.parser')
		univ_info = soup.findAll("a", {"href":re.compile("\/university-entity\/+")})
		if len(univ_info) !=0 :
			for info in univ_info :
				urls_level_1.append('https://www.mooc-list.com'+info['href'])
		else :
			return None

	except AttributeError as e:
		print(e)
		return None 

	return urls_level_1

In [3]:
def get_all_univ_urls() :

	check = True
	urls_level_2 = []
	i = 0
	while check :
		base_url = 'https://www.mooc-list.com/universities-entities?page='+str(i)
		urls_level_1 = extract_univ_urls(base_url)
		if urls_level_1 != None :
			urls_level_2.extend(urls_level_1)
			print(i)
			i = i + 1
		else :
			check = False 
	return urls_level_2

## Given university URLs, let's extract course URLs

In [4]:
def extract_course_urls(univ_url) :

	urls_level_1 = []

	try:
		html_req = Request(univ_url, headers={'User-Agent': 'Mozilla/5.0'})
		html = urlopen(html_req, context=ctx).read()
	except HTTPError as e:
		print(e)
		return None 

	try:
		soup = BeautifulSoup(html, 'html.parser')
		course_info = soup.findAll("div", {"class":"views-field views-field-title"})
		if len(course_info) !=0 :
			for info in course_info :
				that_url = info.find("a", {"href":re.compile("\/course\/+")})
				this_url = 'https://www.mooc-list.com'+that_url['href']
				urls_level_1.append(this_url)
		else :
			return None

	except AttributeError as e:
		print(e)
		return None 

	return urls_level_1

In [5]:
def get_all_course_urls(univ_urls) :
	urls_level_2 = []
	for univ_url in univ_urls :
		check = True
		i = 0
		while check :
			base_url = univ_url+'?page='+str(i)
			urls_level_1 = extract_course_urls(base_url)
			if urls_level_1 != None :
				urls_level_2.extend(urls_level_1)
				print(i)
				i = i + 1
			else :
				check = False
		print(univ_url)        
	return urls_level_2

In [6]:
all_univ_urls = get_all_univ_urls()

0
1
2
3
4
5
6
7
8
9
10
11
12
13


## Now let's pass university URLs to get respective course URLs

In [7]:
# NOTE : This takes lot of time as it needs to loop through every page 
# Please make sure you have uninterrupted internet

all_course_urls = get_all_course_urls(all_univ_urls)

0
https://www.mooc-list.com/university-entity/360training
0
https://www.mooc-list.com/university-entity/4-h-youth-development
0
https://www.mooc-list.com/university-entity/abadir-fine-arts-academy
0
https://www.mooc-list.com/university-entity/abertay-university
0
https://www.mooc-list.com/university-entity/aberystwyth-university
0
https://www.mooc-list.com/university-entity/abu-dhabi-university
0
https://www.mooc-list.com/university-entity/academic-partnerships
0
https://www.mooc-list.com/university-entity/acatech-national-academy-science-and-engineering
0
https://www.mooc-list.com/university-entity/accenture
0
https://www.mooc-list.com/university-entity/accreditrust-llc
0
https://www.mooc-list.com/university-entity/acf-international-action-against-hunger
0
1
2
https://www.mooc-list.com/university-entity/acumen
0
https://www.mooc-list.com/university-entity/addleshaw-goddard
0
https://www.mooc-list.com/university-entity/adelaidex
0
https://www.mooc-list.com/university-entity/adepicos
0


## There's possibility of duplicate URLs, let's convert it into a set

In [8]:
# Sometimes, the courses provided by multiple universities are repeated 

all_course_urls_set = set(all_course_urls)
len(all_course_urls_set)

7974

## Let's save these URLs and dump it as pickle file

In [9]:
# RENAME the FILE based on the date you are extracting -- course_list_YEARMONTHDATE.pkl

with open('course_list_20181029.pkl', 'wb') as f:
    pickle.dump(all_course_urls_set, f)